<a href="https://colab.research.google.com/github/duketran1996/NYC-Crime/blob/main/nyc_crime_datacleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install openclean.**

In [116]:
pip install openclean-core

     |████████████████████████████████| 645kB 5.7MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


**Loading the data file using Socrata. The file is NYPD Arrest Data (Year to Date) as listed below.**



In [139]:
import gzip
import os

from openclean.data.source.socrata import Socrata

datafile = './uip8-fykc.tsv.gz'

# Download file only if it does not exist already.
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        ds = Socrata().dataset('uip8-fykc')
        print('Downloading ...\n')
        print(ds.name + '\n')
        print(ds.description)
        ds.write(f)

**Important import. Run before executing the rest**

In [140]:
from openclean.cluster.knn import knn_clusters, knn_collision_clusters
from openclean.function.similarity.base import SimilarityConstraint
from openclean.function.similarity.text import LevenshteinDistance
from openclean.function.token.ngram import NGrams
from openclean.function.value.threshold import GreaterThan

from openclean.operator.transform.update import update

**Data study: List number of columns.**

In [141]:
from openclean.pipeline import stream

ds = stream(datafile)


print('Schema\n------')
for col in ds.columns:
    print("  '{}'".format(col))
    
print('\n{} rows.'.format(ds.count()))
print("There are {} rows and {} columns in the dataset.".format(ds.count(),len(ds.columns)))

Schema
------
  'ARREST_KEY'
  'ARREST_DATE'
  'PD_CD'
  'PD_DESC'
  'KY_CD'
  'OFNS_DESC'
  'LAW_CODE'
  'LAW_CAT_CD'
  'ARREST_BORO'
  'ARREST_PRECINCT'
  'JURISDICTION_CODE'
  'AGE_GROUP'
  'PERP_SEX'
  'PERP_RACE'
  'X_COORD_CD'
  'Y_COORD_CD'
  'Latitude'
  'Longitude'
  'New Georeferenced Column'

140413 rows.
There are 140413 rows and 19 columns in the dataset.


**Convert to data frame for fixing issues.**

In [142]:
fix = ds.to_df()

**Data study: Profile a sample of 10000 data to detect issues.**

In [143]:
from openclean.profiling.column import DefaultColumnProfiler

profiles = ds.sample(n=10000, random_state=42).profile(default_profiler=DefaultColumnProfiler)

In [144]:
profiles.stats()

,total,empty,distinct,uniqueness,entropy
ARREST_KEY,10000,0,10000,1.000000,13.287712
ARREST_DATE,10000,0,366,0.036600,8.400455
PD_CD,10000,3,177,0.017705,5.371204
PD_DESC,10000,3,170,0.017005,5.340061
KY_CD,10000,3,62,0.006202,4.507652
OFNS_DESC,10000,3,55,0.005502,4.304735
LAW_CODE,10000,0,434,0.043400,6.283008
LAW_CAT_CD,10000,99,4,0.000404,1.042850
ARREST_BORO,10000,0,5,0.000500,2.165878
ARREST_PRECINCT,10000,0,77,0.007700,6.092032


**Data Issues: In OFNS_DESC column, there are many repeated and miss spelling that needs to merge and fix. The impact with this change is later we would like to catergories offenses and have statistics on it.**

In [145]:
offense = ds.select('OFNS_DESC').distinct()

clusters = knn_clusters(
  values=offense,
  sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.7)),
  tokenizer=NGrams(n=4),
  minsize=2
)

for i in clusters:
  print(i)

Cluster({'OFFENSES AGAINST PUBLIC ADMINI': 4925, 'OFFENSES AGAINST PUBLIC SAFETY': 84})
Cluster({'KIDNAPPING & RELATED OFFENSES': 44, 'KIDNAPPING AND RELATED OFFENSES': 1})
Cluster({'INTOXICATED & IMPAIRED DRIVING': 2299, 'INTOXICATED/IMPAIRED DRIVING': 346})
Cluster({'OTHER STATE LAWS (NON PENAL LA': 536, 'OTHER STATE LAWS (NON PENAL LAW)': 5})
Cluster({'ADMINISTRATIVE CODE': 126, 'ADMINISTRATIVE CODES': 1})


In [146]:
offense = ds.select(['OFNS_DESC']).distinct()

kid_val = []

for i in offense:
  if 'KIDNAPPING' in i:
    kid_val.append(i)
    print(i)

KIDNAPPING & RELATED OFFENSES
KIDNAPPING AND RELATED OFFENSES
KIDNAPPING


**Data fix: Change KIDNAPPING and KIDNAPPING & RELATED OFFENSES to KIDNAPPING AND RELATED OFFENSES**

In [147]:
kid_dict = {
    'KIDNAPPING & RELATED OFFENSES': 'KIDNAPPING AND RELATED OFFENSES',
    'KIDNAPPING': 'KIDNAPPING AND RELATED OFFENSES'
}

fix = update(fix, columns='OFNS_DESC', func=kid_dict)

check_fix = fix.loc[fix['OFNS_DESC'].isin(kid_val)]
check_fix['OFNS_DESC'].unique()

array(['KIDNAPPING AND RELATED OFFENSES'], dtype=object)

In [148]:
offense = ds.select('OFNS_DESC').distinct()
administrative_val = []
for i in offense:
  if 'ADMINISTRATIVE' in i:
    administrative_val.append(i)
    print(i)

ADMINISTRATIVE CODES
ADMINISTRATIVE CODE


**Data fix: Change ADMINISTRATIVE CODES to ADMINISTRATIVE CODE**

In [149]:
ad_dict = {
    'ADMINISTRATIVE CODES': 'ADMINISTRATIVE CODE',
}

fix = update(fix, columns='OFNS_DESC', func=ad_dict)

check_fix = fix.loc[fix['OFNS_DESC'].isin(administrative_val)]
check_fix['OFNS_DESC'].unique()

array(['ADMINISTRATIVE CODE'], dtype=object)

In [150]:
offense = ds.select('OFNS_DESC').distinct()
tox_val = []
for i in offense:
  if 'INTOXICATED' in i:
    tox_val.append(i)
    print(i)

INTOXICATED & IMPAIRED DRIVING
INTOXICATED/IMPAIRED DRIVING


**Data fix: Change INTOXICATED & IMPAIRED DRIVING, INTOXICATED/IMPAIRED DRIVING to INTOXICATED AND IMPAIRED DRIVING**

In [151]:
tox_dict = {
    'INTOXICATED & IMPAIRED DRIVING': 'INTOXICATED AND IMPAIRED DRIVING',
    'INTOXICATED/IMPAIRED DRIVING': 'INTOXICATED AND IMPAIRED DRIVING'
}

fix = update(fix, columns='OFNS_DESC', func=tox_dict)

check_fix = fix.loc[fix['OFNS_DESC'].isin(['INTOXICATED AND IMPAIRED DRIVING'] + tox_val)]
check_fix['OFNS_DESC'].unique()

array(['INTOXICATED AND IMPAIRED DRIVING'], dtype=object)

**Data issues: The borough of NYC that the arrest happen. The data K,M,B,Q,S is unclear to us.**

In [152]:
# Print the values in decreasing order of frequency.

states = ds.distinct('ARREST_BORO')
for rank, val in enumerate(states.most_common()):
    st, freq = val
    print('{:<3} {}  {:>10}'.format('{}.'.format(rank + 1), st, '{:,}'.format(freq)))

1.  K      38,258
2.  M      33,255
3.  B      32,724
4.  Q      29,981
5.  S       6,195


**Data fix: Change ambiguous abbreviation of column ARREST_BORO to full form.**

In [153]:
boro_dict = {
    'B': 'Bronx',
    'S': 'Staten Island',
    'K': 'Brooklyn',
    'M': 'Manhattan',
    'Q': 'Queens',
}

fix = update(fix, columns='ARREST_BORO', func=boro_dict)

fix['ARREST_BORO'].unique()

array(['Bronx', 'Queens', 'Manhattan', 'Brooklyn', 'Staten Island'],
      dtype=object)

**Data issues: The columns PERP_SEX and LAW_CAT_CD are also having values that is easier to read if written in full text instead of abbreviation.**

In [154]:
sex = ds.distinct('PERP_SEX')

for i in sex:
  print(i)

M
F


In [155]:
law_cat_cd = ds.distinct('LAW_CAT_CD')

for i in law_cat_cd:
  print(i)

F
M

I
V


**Data fix: Change abbreviation of LAW_CAT_CD to long form.**

In [156]:
law_cat_cd_dict = {
    'F': 'Felony',
    'M': 'Misdemeanor',
    'V': 'Violation',
    'I': 'Traffic Infraction',
    '': 'Unknown'
}

fix = update(fix, columns='LAW_CAT_CD', func=law_cat_cd_dict)

fix['LAW_CAT_CD'].unique()

array(['Felony', 'Misdemeanor', 'Unknown', 'Traffic Infraction',
       'Violation'], dtype=object)

**Data fix: Change abbreviation of PERP_SEX to long form.**

In [157]:
perp_sex_dict = {
    'F': 'Female',
    'M': 'Male',
}

fix = update(fix, columns='PERP_SEX', func=perp_sex_dict)

fix['PERP_SEX'].unique()

array(['Male', 'Female'], dtype=object)

**Data issues: There are unnecessary columns in our dataset that we don't care about such as X_COORD_CD and Y_COORD_CD which list midblock X and Y-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104)**

In [158]:
x = ds.distinct('X_COORD_CD')
y = ds.distinct('Y_COORD_CD')

print(x)
print(y)

Counter({'1017119': 1176, '1026486': 984, '1005041': 979, '962822': 925, '1007694': 868, '1046405': 831, '1011811': 826, '1011751': 802, '1057767': 793, '1017440': 774, '1006537': 741, '1041879': 738, '1020183': 732, '1008691': 710, '997897': 698, '1032084': 665, '999507': 661, '942825': 652, '1031076': 635, '988972': 617, '1007128': 607, '988211': 596, '1036628': 571, '987220': 561, '1003508': 560, '1018714': 558, '982746': 551, '1020237': 519, '1008228': 517, '1027430': 515, '998829': 514, '998016': 511, '999428': 510, '994297': 495, '1028605': 494, '1053650': 491, '1013086': 490, '991591': 474, '1013787': 470, '1001889': 464, '988897': 463, '984058': 458, '1000598': 437, '990824': 425, '985802': 417, '988702': 409, '996343': 404, '995118': 399, '1046315': 391, '1032140': 373, '982351': 365, '984686': 362, '1005312': 357, '987633': 355, '992254': 349, '1000521': 344, '954801': 339, '1002162': 334, '984984': 333, '1010576': 325, '983673': 322, '983903': 318, '999005': 317, '1010984': 

**Data fix: Our solution is to drop the columns.**

In [159]:
fix  = fix.drop(columns=['X_COORD_CD', 'Y_COORD_CD'])

check_fix = fix.columns.values.tolist()

print(check_fix)

['ARREST_KEY', 'ARREST_DATE', 'PD_CD', 'PD_DESC', 'KY_CD', 'OFNS_DESC', 'LAW_CODE', 'LAW_CAT_CD', 'ARREST_BORO', 'ARREST_PRECINCT', 'JURISDICTION_CODE', 'AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'Latitude', 'Longitude', 'New Georeferenced Column']


**Data issues: Even the data description states the PD_CD supposed to have 3 digit number. We found there are values with 2 digits. Howeve, we found no issue with KY_CD as these 2 columns have similar requirements.**

In [160]:
pd_cd = ds.distinct('PD_CD')

for i in ky_cd:
  if len(i) != 3:
    print(i)


49
16
0
15
35
29
30


**Data fix: Further study is needed for the fix as we don't know how PD_CD is related to the rest of the data set.**

**Data issues: The data ASIAN / PACIFIC ISLANDER is better to be fix by removing space between / for easier comparison for analysis later on.**

In [161]:
race = ds.distinct('PERP_RACE')

for i in race:
  print(i)

BLACK
WHITE
WHITE HISPANIC
BLACK HISPANIC
ASIAN / PACIFIC ISLANDER
UNKNOWN
AMERICAN INDIAN/ALASKAN NATIVE


**Data fix: Remove space between ASIAN / PACIFIC ISLANDER.**

In [162]:
race_dict = {
    'ASIAN / PACIFIC ISLANDER': 'ASIAN/PACIFIC ISLANDER',
    'M': 'Male',
}

fix = update(fix, columns='PERP_RACE', func=race_dict)

fix['PERP_RACE'].unique()

array(['BLACK', 'WHITE', 'WHITE HISPANIC', 'BLACK HISPANIC',
       'ASIAN/PACIFIC ISLANDER', 'UNKNOWN',
       'AMERICAN INDIAN/ALASKAN NATIVE'], dtype=object)

**Data issues: Found new issues with PD_DESC. Some spellings are incorrect. This also impacts as we want to catergorize the PD description to compare with the offense description.**

In [163]:
pd = ds.select('PD_DESC').distinct()

clusters = knn_clusters(
  values=pd,
  sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.9)),
  tokenizer=NGrams(n=4),
  minsize=2
)

for i in clusters:
  print(i)

Cluster({'NY STATE LAWS,UNCLASSIFIED MIS': 428, 'NY STATE LAWS,UNCLASSIFIED VIO': 262})
Cluster({'AGGRAVATED HARASSMENT 2': 2365, 'AGGRAVATED HARASSMENT 1': 32})
Cluster({'IMPRISONMENT 2,UNLAWFUL': 83, 'IMPRISONMENT 1,UNLAWFUL': 29})
Cluster({'RECKLESS ENDANGERMENT 1': 647, 'RECKLESS ENDANGERMENT 2': 608})
Cluster({'CONTROLLED SUBSTANCE,INTENT TO': 2400, 'CONTROLLED SUBSTANCE, INTENT T': 362})
Cluster({'CONTROLLED SUBSTANCE, POSSESSI': 4773, 'CONTROLLED SUBSTANCE,POSSESS.': 474})
Cluster({'CONTROLLED SUBSTANCE,SALE 3': 976, 'CONTROLLED SUBSTANCE, SALE 5': 67, 'CONTROLLED SUBSTANCE,SALE 2': 66, 'CONTROLLED SUBSTANCE,SALE 1': 56, 'CONTROLLED SUBSTANCE, SALE 4': 23})
Cluster({'PROSTITUTION 2, UNDER 16': 4, 'PROSTITUTION 1, UNDER 11': 1})
Cluster({'UNAUTHORIZED USE VEHICLE 3': 522, 'UNAUTHORIZED USE VEHICLE 2': 241})
Cluster({'IMPAIRED DRIVING,DRUG': 61, 'IMPAIRED DRIVING, DRUGS': 14})
Cluster({'TRESPASS 3, CRIMINAL': 1049, 'TRESPASS 2, CRIMINAL': 503})
Cluster({'CUSTODIAL INTERFERENCE 2':

**Only miss spelling one is considered to be fixed such as: CONTROLLED SUBSTANCE, POSSESSI, CONTROLLED SUBSTANCE, INTENT T, CONTROLLED SUBSTANCE,POSSESS., and spaces between CONTROLLED SUBSTANCE, SALE**

In [164]:
pd_desc = ds.select('PD_DESC').distinct()

for i in pd_desc:
  if 'CONTROLLED SUBSTANCE,' in i:
    print(i)

CONTROLLED SUBSTANCE, POSSESSI
CONTROLLED SUBSTANCE,INTENT TO
CONTROLLED SUBSTANCE, INTENT T
CONTROLLED SUBSTANCE,SALE 3
CONTROLLED SUBSTANCE, SALE 5
CONTROLLED SUBSTANCE,POSSESS.
CONTROLLED SUBSTANCE,SALE 1
CONTROLLED SUBSTANCE, SALE 4
CONTROLLED SUBSTANCE,SALE 2


**Data fix: Change spacing in SALE and spelling to POSSESSION**



In [165]:
control_dict = {
    'CONTROLLED SUBSTANCE, POSSESSI': 'CONTROLLED SUBSTANCE, POSSESSION',
    'CONTROLLED SUBSTANCE,POSSESS.': 'CONTROLLED SUBSTANCE, POSSESSION',
    'CONTROLLED SUBSTANCE,INTENT TO': 'CONTROLLED SUBSTANCE, INTENT',
    'CONTROLLED SUBSTANCE, INTENT T': 'CONTROLLED SUBSTANCE, INTENT',
    'CONTROLLED SUBSTANCE,SALE 3': 'CONTROLLED SUBSTANCE, SALE 3',
    'CONTROLLED SUBSTANCE,SALE 1': 'CONTROLLED SUBSTANCE, SALE 1',
    'CONTROLLED SUBSTANCE,SALE 2': 'CONTROLLED SUBSTANCE, SALE 2'
}

fix = update(fix, columns='PD_DESC', func=control_dict)


check_fix = fix[fix['PD_DESC'].str.contains('CONTROLLED SUBSTANCE')]
check_fix['PD_DESC'].unique()

array(['CONTROLLED SUBSTANCE, POSSESSION', 'CONTROLLED SUBSTANCE, INTENT',
       'CONTROLLED SUBSTANCE, SALE 3', 'CONTROLLED SUBSTANCE, SALE 5',
       'CONTROLLED SUBSTANCE, SALE 1', 'CONTROLLED SUBSTANCE, SALE 4',
       'CONTROLLED SUBSTANCE, SALE 2'], dtype=object)

**Only fix DRUG spelling**

In [166]:
pd_desc = ds.select('PD_DESC').distinct()

for i in pd_desc:
  if 'IMPAIRED DRIVING' in i:
    print(i)

IMPAIRED DRIVING,DRUG
IMPAIRED DRIVING / ALCOHOL
IMPAIRED DRIVING, DRUGS


**Data fix: Fix DRUG spelling.**

In [167]:
impair_dict = {
    'IMPAIRED DRIVING, DRUGS': 'IMPAIRED DRIVING,DRUG'
}

fix = update(fix, columns='PD_DESC', func=impair_dict)

check_fix = fix.loc[fix['PD_DESC'].isin(['IMPAIRED DRIVING, DRUGS', 'IMPAIRED DRIVING,DRUG'])]

check_fix['PD_DESC'].unique()

array(['IMPAIRED DRIVING,DRUG'], dtype=object)

**Data study: Perform a scan to check date format. No issues found.**

In [168]:
date = ds.distinct('ARREST_DATE')

import datetime 
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%m/%d/%Y')
    except ValueError:
        print(date_text)
        #raise ValueError("Incorrect data format, should be YYYY-MM-DD")

for i in date:
  validate(i)

**Finalize data set: Save data clean file to csv file for analysis.**

In [169]:
fix.to_csv(r'arrest_data_clean.csv')